<a href="https://colab.research.google.com/github/idAryan/ML-Projects/blob/main/Fake%20News%20Prediction/Fake_News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

About the dataset:
id: Unique id for news article
title: the title of news article
author: author of the news article
text: the text article could be incomplete
label: a label that marks whether the news article is real or fake


1: fake news

2: real news

Importing the dependencies

In [37]:
import numpy as np
import pandas as pd
import re
import nltk.corpus
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [38]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [39]:
# printing the stop words in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-Processing

In [40]:
import pandas as pd

# Skip bad lines and print a warning
news_dataset = pd.read_csv('/content/train.csv', encoding='iso-8859-1', on_bad_lines='warn')

Skipping line 4769: expected 5 fields, saw 7
Skipping line 20203: expected 5 fields, saw 8
Skipping line 20206: expected 5 fields, saw 6
Skipping line 20225: expected 5 fields, saw 8
Skipping line 20227: expected 5 fields, saw 8
Skipping line 20235: expected 5 fields, saw 7
Skipping line 20236: expected 5 fields, saw 6
Skipping line 23952: expected 5 fields, saw 11
Skipping line 23953: expected 5 fields, saw 7
Skipping line 23955: expected 5 fields, saw 13
Skipping line 23957: expected 5 fields, saw 6
Skipping line 23958: expected 5 fields, saw 7
Skipping line 23960: expected 5 fields, saw 7
Skipping line 23963: expected 5 fields, saw 6
Skipping line 23968: expected 5 fields, saw 15
Skipping line 23973: expected 5 fields, saw 12
Skipping line 23974: expected 5 fields, saw 10
Skipping line 23977: expected 5 fields, saw 6
Skipping line 23983: expected 5 fields, saw 6
Skipping line 23992: expected 5 fields, saw 7
Skipping line 23996: expected 5 fields, saw 6
Skipping line 23998: expected 

In [41]:
news_dataset.shape

(32949, 5)

In [42]:
# print the first 5 rows of the dataframe
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [43]:
# counting the number of missing value in the dataset
news_dataset.isnull().sum()

id           0
title      949
author    3192
text       192
label      148
dtype: int64

In [44]:
# replacing the null values with empty string
news_dataset=news_dataset.fillna('')

In [45]:
 # merging the author name and news title
 news_dataset['content']=news_dataset['author']+' '+news_dataset['title']

In [46]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didnât Even...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
32944    Jerome Hudson Rapper T.I.: Trump a âPoster C...
32945    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
32946    Michael J. de la Merced and Rachel Abrams Macy...
32947    Alex Ansary NATO, Russia To Hold Parallel Exer...
32948              David Swanson What Keeps the F-35 Alive
Name: content, Length: 32949, dtype: object


In [47]:
# Separating the data and label
X=news_dataset.drop(columns='label',axis=1)
Y=news_dataset['label']

In [48]:
print(X)

          id                                              title  \
0          0  House Dem Aide: We Didnât Even See Comeyâs...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
32944  20795  Rapper T.I.: Trump a âPoster Child For White...   
32945  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
32946  20797  Macyâs Is Said to Receive Takeover Approach ...   
32947  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
32948  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [49]:
print(Y)

0        1
1        0
2        1
3        1
4        1
        ..
32944    0
32945    0
32946    0
32947    1
32948    1
Name: label, Length: 32949, dtype: object


Stemming:
Stemming is the process of reducing a word to its Root word
example
actor,actress,acting -----> act

In [50]:
port_stem=PorterStemmer()

In [57]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)
  return stemmed_content

In [54]:
news_dataset

,id,title,author,text,label,content
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1,Darrell Lucus House Dem Aide: We Didnât Even...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...
...,...,...,...,...,...,...
32944,20795,Rapper T.I.: Trump a âPoster Child For White...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,Jerome Hudson Rapper T.I.: Trump a âPoster C...
32945,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma..."
32946,20797,Macyâs Is Said to Receive Takeover Approach ...,Michael J. de la Merced and Rachel Abrams,The Macyâs of today grew from the union of s...,0,Michael J. de la Merced and Rachel Abrams Macy...
32947,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"Alex Ansary NATO, Russia To Hold Parallel Exer..."


In [58]:
news_dataset['content']=news_dataset['content'].apply(stemming)

In [59]:
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
32944    jerom hudson rapper trump poster child white s...
32945    benjamin hoffman n f l playoff schedul matchup...
32946    michael j de la merc rachel abram maci said re...
32947    alex ansari nato russia hold parallel exercis ...
32948                            david swanson keep f aliv
Name: content, Length: 32949, dtype: object


In [68]:
# Separating the data & label
X=news_dataset['content'].values
Y=news_dataset['label'].values

In [61]:
print(X)
print(Y)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']
['1' '0' '1' ... '0' '1' '1']


In [63]:
Y.shape

(32949,)

In [69]:
# Converting the textual data to numerical data
#Term frequency Inverse document frequency: count the words number of times it is reapeating
vectorizer=TfidfVectorizer()
vectorizer.fit(X)

X=vectorizer.transform(X)

In [70]:
print(X)

  (0, 15738)	0.28391081883521274
  (0, 13517)	0.25631818216737756
  (0, 8939)	0.36831100003241907
  (0, 8659)	0.2924113140385708
  (0, 7717)	0.24666544126914805
  (0, 7026)	0.21738343549365183
  (0, 4988)	0.2320711759689065
  (0, 3802)	0.27021145201700636
  (0, 3610)	0.36300903656502076
  (0, 2967)	0.2469863550865947
  (0, 2489)	0.36300903656502076
  (0, 267)	0.27021145201700636
  (1, 16857)	0.30282942763171056
  (1, 6837)	0.1909234544932658
  (1, 5521)	0.7124554218823693
  (1, 3578)	0.2631670355142566
  (1, 2821)	0.1913808662470053
  (1, 2229)	0.3827363199698232
  (1, 1899)	0.15616838944185288
  (1, 1502)	0.2957882064641894
  (2, 15663)	0.4142311242893174
  (2, 9652)	0.4922170234571005
  (2, 5986)	0.34928382672781505
  (2, 5406)	0.3882287560590808
  (2, 3111)	0.46056252660443764
  :	:
  (32946, 13166)	0.24416289043811096
  (32946, 12387)	0.2729344636973869
  (32946, 12180)	0.24890112590665506
  (32946, 10340)	0.08088494443431321
  (32946, 9620)	0.17509172338402343
  (32946, 9549)	0.29

Splitting the dataset to training & test data

In [72]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y, test_size=0.2,random_state=2)

Training the Model: Logistic Regression Model

In [73]:
model=LogisticRegression()
model.fit(X_train,Y_train)

LogisticRegression()

In [79]:
# Evaluation
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction,Y_train)
print('Accuracy score on training data is : ', training_data_accuracy)

Accuracy score on training data is :  0.988808376645548


In [80]:
# Evaluation on test data
X_test_prediction=model.predict(X_test)
test_data_accuracy=accuracy_score(X_test_prediction,Y_test)
print('Accuracy on test data is : ',test_data_accuracy)

Accuracy on test data is :  0.985432473444613


Making a Predictive System

In [83]:
X_new=X_test[3]
prediction=model.predict(X_new)
print(prediction)

if(prediction[0]==0):
  print('The news is real')
else:
  print('The news is false')

['1']
The news is false


In [85]:
print(Y_test[3])

1
